<a href="https://colab.research.google.com/github/LeeHyungDo/Mongotest/blob/master/HPC_BM25_RAG%EC%8B%A4%EC%8A%B5%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BM25 Retriever RAG 실습

## Setup

In [ ]:
%pip install llama-index
%pip install llama-index-retrievers-bm25
%pip install llama-index-llms-vllm
%pip install datasets==3.5.0
%pip install vllm

In [ ]:
import os

os.environ["HF_HOME"] = "model/"

from llama_index.core import Settings
from llama_index.llms.vllm import Vllm

Settings.llm = Vllm(
    dtype='float16',
    model='LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct'
)

INFO 05-23 06:59:48 [__init__.py:239] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

INFO 05-23 06:59:55 [config.py:2968] Downcasting torch.float32 to torch.float16.
INFO 05-23 07:00:09 [config.py:717] This model supports multiple tasks: {'reward', 'generate', 'classify', 'score', 'embed'}. Defaulting to 'generate'.
WARNING 05-23 07:00:09 [arg_utils.py:1658] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 05-23 07:00:09 [llm_engine.py:240] Initializing a V0 LLM engine (v0.8.5.post1) with config: model='LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct', speculative_config=None, tokenizer='LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig

tokenizer_config.json:   0%|          | 0.00/70.7k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/22.2k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 05-23 07:00:50 [loader.py:458] Loading weights took 2.33 seconds
INFO 05-23 07:00:50 [model_runner.py:1140] Model loading took 4.5146 GiB and 33.788425 seconds
INFO 05-23 07:00:57 [worker.py:287] Memory profiling takes 6.41 seconds
INFO 05-23 07:00:57 [worker.py:287] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 05-23 07:00:57 [worker.py:287] model weights take 4.51GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 1.95GiB; the rest of the memory reserved for KV Cache is 6.76GiB.
INFO 05-23 07:00:58 [executor_base.py:112] # cuda blocks: 5906, # CPU blocks: 3495
INFO 05-23 07:00:58 [executor_base.py:117] Maximum concurrency for 32768 tokens per request: 2.88x
INFO 05-23 07:01:01 [model_runner.py:1450] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. 

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

INFO 05-23 07:01:36 [model_runner.py:1592] Graph capturing finished in 35 secs, took 0.21 GiB
INFO 05-23 07:01:36 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 45.50 seconds


## Load Data


In [ ]:
from llama_index.core import Document
from datasets import load_dataset

In [ ]:
dataset = load_dataset('heegyu/kowikitext', trust_remote_code=True, split='train[:1000]')

README.md:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

kowikitext.py:   0%|          | 0.00/5.70k [00:00<?, ?B/s]

kowikitext-20221001.parquet:   0%|          | 0.00/498M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['id', 'revid', 'url', 'title', 'text'],
    num_rows: 1000
})

In [ ]:
print(dataset['title'][:50])

['지미 카터', '수학', '수학 상수', '문학', '나라 목록', '화학', '체첸 공화국', '맥스웰 방정식', '초월수', '음계', '대한민국 제16대 대통령 선거', '함석헌', '백남준', '2002년', '12월 19일', '5월 31일', '6월 30일', '우크라이나', '가위', '위키', '지구과학', '새천년 민주당', '아오조라 문고', '프로젝트 구텐베르크', '겐지모노가타리', '귄터 그라스', '파이어아벤트', '일반 상대성이론', '아인슈타인', '데니스 리치', '주기율표', '아미노산', '히라가나', '나라 이름순 수도 목록', '토마스 만', '하인리히 뵐', '방정식', '삼각함수 항등식', '노무현', '곱셈적 함수', '체비쇼프 다항식', '파이의 날', '코사인 법칙', '사인 법칙', '벡터 공간', '펜로즈 삼각형', '수론적 함수', '물리 상수', '대수학의 기본 정리', '정규 분포']


In [ ]:
documents = [Document(text=row['text']) for row in dataset]

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

# initialize node parser
splitter = SentenceSplitter(chunk_size=512)
##위키피디아 문서 길면 잘라서 관리 한 문서가 여려개 쪼개져 저장될수 잇음

nodes = splitter.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/1000 [00:00<?, ?it/s]

## BM25 Retriever + Disk Persistance

In [ ]:
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer

# We can pass in the index, docstore, or list of nodes to create the retriever
bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=3,
)

DEBUG:bm25s:Building index from IDs objects


In [ ]:
bm25_retriever.persist("./bm25_retriever") #인덱싱 정보 저장

loaded_bm25_retriever = BM25Retriever.from_persist_dir("./bm25_retriever")#저장 정보 불러오기

Finding newlines for mmindex:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

In [ ]:
from llama_index.core.response.notebook_utils import display_source_node

retrieved_nodes = bm25_retriever.retrieve("백남준은 누구인가요?")
for node in retrieved_nodes:
    display_source_node(node, source_length=512)

**Node ID:** c356b546-b52e-449e-a722-61221c94d731<br>**Similarity:** 3.9395880699157715<br>**Text:** 또한 백남준은 같은 해 베니스 비엔날레 국가전시관 부문에 한국관을 설치하는 일에 결정적인 역할을 했다. 이로써 한국 미술이 세계 미술계에 진출하는 교두보가 마련되었다고 하겠다. 같은 해 그의 예술적 정수가 담긴 일렉트로닉 수퍼하이웨이 전시를 진행했다. 1996년 4월 9일 뇌졸중으로 쓰러졌으며, 6개월만인 그해 10월에 재기했다. 2000년 뉴욕 구겐하임 미술관에서 ‘백남준의 세계’ 라는 대규모 회고전이 열렸으며, 이때 백남준은 레이저 아트 ‘야곱의 사다리’, ‘삼원소’ 등을 전시한 바 있다.
2006년 1월 29일, 미국 마이애미의 자택에서 노환으로 75세로 별세, 유해가 서울, 뉴욕, 독일에 나눠서 안치되었다..<br>

**Node ID:** 91ad3937-9b24-4681-8054-50d66585e8b2<br>**Similarity:** 3.8545095920562744<br>**Text:** 이 영감은 "세계의 역사는 우리에게 알려준다. 주어진 게임에서 이길 수 없다면 규칙을 바꿔라"라는 것으로 규정된다. 이후 1950년대부터 활발해지기 시작한 독일 라인 지역의 액션뮤직의 현장에서 백남준은 ‘아시아에서 온 문화테러리스트’(앨런 카프로)라고 불릴 정도의 탁월한 퍼포먼스 아티스트로 활약했다. 1959년 ‘존 케이지에게 보내는 경의’에서 음악적 콜라주와 함께 피아노를 부수는 퍼포먼스를 선보이는 것을 시작으로, 바이올린을 단숨에 파괴하거나(바이올린 솔로) 존 케이지가 착용한 넥타이를 잘라버리는 퍼포먼스(피아노 포르테를 위한 연습곡)가 특히 유명하다. 이 초기 퍼포먼스에 대해 백남준은 스스로 "충격, 표현주의, 낭만주의, 클라이맥스, 놀라움, 기타 등등을 보여준 것"이라고 표현한 바 있다.<br>

**Node ID:** 372e1d22-bb30-47e1-9706-3e9da2e6937f<br>**Similarity:** 3.644620656967163<br>**Text:** 졸업 논문은 ‘아르놀트 쇤베르크 연구’이다.
1956년 백남준은 졸업과 함께 독일로 유학을 떠나 뮌헨 대학교 및 쾰른 대학교 등에서 서양의 건축, 음악사, 철학 등을 공부하였다. 뮌헨 대학교 입학 1년 후에는 프라이부르크 국립 음악 대학교로 옮겨 볼프강 포르트너 교수에게 배우지만, 곧 쇤베르크 이후 현대음악의 실험이 활발히 진행되던 다름슈타트 하기 강좌에 참여했다. 1958년 그 곳에서 현대음악가 존 케이지를 만나 그의 음악에 대한 파괴적 접근과 자유정신으로부터 깊은 영감을 얻었다. 이 영감은 "세계의 역사는 우리에게 알려준다. 주어진 게임에서 이길 수 없다면 규칙을 바꿔라"라는 것으로 규정된다. 이후 1950년대부터 활발해지기 시작한 독일 라인 지역의 액션뮤직의 현장에서 백남준은 ‘아시아에서 온 문화테러리스트’(앨런 카프로)라고 불릴 정도의 탁월한 퍼포먼스 아티스트로 활약했다.<br>

In [ ]:
retrieved_nodes = bm25_retriever.retrieve("맥스웰 방정식이 무엇인지 설명해주세요.")
for node in retrieved_nodes:
    display_source_node(node, source_length=5000)

**Node ID:** 8ec82861-446b-4de8-9ba4-5f15f53952e8<br>**Similarity:** 6.281866073608398<br>**Text:** 자기에서 전기가 유도될 수 있다는 것이 알려지자, 이것을 이용한 많은 기술들이 발명되었다. 니콜라 테슬라는 전자기 유도를 이용하여 교류 발전기를 발명하였다.
맥스웰 방정식.
1864년 제임스 클러크 맥스웰은 기존의 전기와 자기에 대한 이론의 수리 모형들을 약 20개의 방정식으로 정리하였다. 맥스웰은 나중에 의미가 같은 방정식들은 과감히 생략하고 본질적인 네 개의 방정식으로 정리하였다. 이를 맥스웰 방정식이라고 한다.
맥스웰 방정식이 다루는 전자기학의 법칙은 다음과 같다.
맥스웰은 전자기학에 대한 법칙들의 수리 모형을 정리하면서 로런츠 힘은 다루지 않았다. 로렌츠 힘은 전하를 띈 물체가 전기장 안에서 받는 힘을 뜻한다. 오늘날 전자기학은 맥스웰 방정식과 로런츠 힘을 기반으로 하고 있다.
맥스웰은 맥스웰 방정식을 정리한 후, 자계가 사인파와 같은 파동을 보이며 변화한다면 전계 역시 그에 따라 사인파 파동을 만들어 내게 될 것이고, 그 반대도 마찬가지 일 것이므로, 전기와 자기의 성질을 모두 갖는 파동, 즉 전자파가 존재할 것이라고 예측하였다.<br>

**Node ID:** c88f590b-1cb3-4f85-90d6-ffdda8f7505e<br>**Similarity:** 5.927718162536621<br>**Text:** 그러나 아인슈타인은 사이언스에의 기고문에서 이를 "맥스웰 방정식"이라 부르며, 이 방정식들이 이론물리학의 기초라고 설명하였다. 맥스웰은 방정식을 정리하면서 헤비사이드의 전위와 벡터 위치 등 위치 요소를 중요한 개념으로 도입하였다. 1884년 맥스웰은 전자기파의 전달을 중력과 같이 원격에서 상호작용하는 힘이 아닌 전자기장에서 빛의 속도로 전파되는 전위로 파악하였다. 라디오 안테나에 대한 현대의 분석에서도 맥스웰의 백터와 스칼라 위키에 대한 수식만으로 서로 떨어져 있는 안테나 사이에 작용하는 전파의 영향을 모두 설명할 수 있다.
맥스웰 방정식과 관련한 헤비사이드의 업적은 맥스웰이 여러 논문과 책에서 서술한 맥스웰 방정식을 오늘날과 같은 4개의 방정식으로 정리하였다는 것이다.
《물리적 역선에 대해》 (1861년).
오늘날 4개의 방정식으로 정리된 맥스웰의 방정식은 1861년 발표된 논문인 《물리적 역선에 대해》에 기반한 것이다. 이 논문에는 전자기장에 대한 다수의 방정식이 실려있다.<br>

**Node ID:** 25094d67-dc1d-4a31-a221-00214a3127dc<br>**Similarity:** 5.395963668823242<br>**Text:** 맥스웰 방정식과 관련한 헤비사이드의 업적은 맥스웰이 여러 논문과 책에서 서술한 맥스웰 방정식을 오늘날과 같은 4개의 방정식으로 정리하였다는 것이다.
《물리적 역선에 대해》 (1861년).
오늘날 4개의 방정식으로 정리된 맥스웰의 방정식은 1861년 발표된 논문인 《물리적 역선에 대해》에 기반한 것이다. 이 논문에는 전자기장에 대한 다수의 방정식이 실려있다.
1855년 맥스웰은 케임브리지 철학 학회에서 《패러데이의 역선》을 발표하면서 formula_8와 formula_9 벡터의 차이점을 설명하였다. 이 논문은 오늘날에도 패러데이 전자기 유도 법칙에 대한 가장 간결한 모형으로 인정받고 있다. 여기서 맥스웰은 전류에 관한 모든 지식을 미분 방정식으로 나타내었다.
1855년 맥스웰이 제안한 분자 와동의 바다란 개념은 1861년 《물리 역선에 대해》에서 보다 분명하게 소개되었다. 이 논문에서는 자기장이 형성되는 분자 규모의 와동에서 formula_8의 밀도에 따라 formula_9의 순 와동 운동이 결정된다고 보았다.<br>

In [ ]:
from llama_index.core import get_response_synthesizer# 문서가져왔으면 토대로 응답생성하기 위한거
from llama_index.core.query_engine import RetrieverQueryEngine

response_synthesizer = get_response_synthesizer(response_mode="refine") #refine 응답 한번 만족 안될수 있음 그다음에 더 정제해달라는 거

query_engine = RetrieverQueryEngine(retriever=bm25_retriever,
                                    response_synthesizer=response_synthesizer)

In [ ]:
prompts = query_engine.get_prompts()

In [ ]:
prompts.keys()

dict_keys(['response_synthesizer:text_qa_template', 'response_synthesizer:refine_template'])

In [ ]:
print(prompts['response_synthesizer:refine_template'].default_template.template)

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


In [ ]:
prompts['response_synthesizer:refine_template'].default_template.template = """원 질의는 다음과 같습니다.: {query_str}
우리에게 주어진 응답은 다음과 같습니다.: {existing_answer}
여기서 아래 컨텍스트를 참고하여, 응답을 더 낫게 만들 수 있는 상황입니다.
------------
{context_msg}
------------
주어진 새로운 컨텍스트에서, 주어진 질의에 더 나은 답을 응답해주세요. 주어진 컨텍스트가 충분히 유용하지 않으면 기존 응답을 내주세요. 모든 응답은 한국어로 해주세요.
개선된 답변: """

In [ ]:
print(prompts['response_synthesizer:refine_template'].default_template.template)

원 질의는 다음과 같습니다.: {query_str}
우리에게 주어진 응답은 다음과 같습니다.: {existing_answer}
여기서 아래 컨텍스트를 참고하여, 응답을 더 낫게 만들 수 있는 상황입니다.
------------
{context_msg}
------------
주어진 새로운 컨텍스트에서, 주어진 질의에 더 나은 답을 응답해주세요. 주어진 컨텍스트가 충분히 유용하지 않으면 기존 응답을 내주세요. 모든 응답은 한국어로 해주세요.
개선된 답변: 


In [ ]:
response = query_engine.query("백남준은 누구인가요?")
print(response)

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


백남준은 1932년 7월 24일에 태어난 한국의 avant-garde 현대 예술가입니다. 그의 예술적 경력은 다음과 같은 핵심 요소로 정의됩니다:

1. **예술적 실험**: 백남준은 특히 전자 음악과 비디오 아트 분야에서 혁신적인 작품들을 선보였습니다. 그의 창작 과정은 종종 기존의 예술적 경계를 무너뜨리고 새로운 형태를 탐구하는 데 초점을 맞추었습니다. 예를 들어, 클래식 악기의 소리를 전자 매체를 통해 재해석하거나, 관객의 참여와 인터랙션을 강조한 퍼포먼스 작품을 통해 관객 경험이 예술 작품 자체의 일부가 되는 방식을 개척했습니다. 'Tetris', 'Spatial Dust', 그리고 'Conduit' 시리즈는 이러한 접근법의 좋은 예시입니다.

2. **문화 간 교류**: 독일과 한국 간의 문화적 다리 역할을 했습니다. 독일의 현대 예술 환경 속에서 활동하면서, 그는 동서양의 미술적 전통과 현대성의 융합을 주도했습니다. 이러한 노력으로 그는 한국의 문화적 표현이 국제적으로 인정받는 데 결정적인 역할을 했습니다. 특히 뮌헨 대학교와 쾰른 대학교에서의 학문적 배경이 이러한 문화적 교류를 위한 감각과 이해를 제공했습니다.

3. **미디어 아티스트로서의 입지**: 유엔교육문화기구 (UNESCO)의 다양한 프로젝트에 참여하며 미디어 아트의 선구자적 인물로 자리잡았습니다. MTV의 심사위원이던 최onasyn과 함께 비디오 아트의 주류 진입을 이끌기도 했습니다. 켈리 데뷔를 한 "Bonjour Sweet Day"는 그의 미디어 아트 실험을 상징적으로 잘 보여주는 작품 중 하나입니다.

백남준은 2008년에 세상을 떠났지만, 그의 예술적 Vision과 혁신적인 실험 정신은 오늘날까지 디지털 아트와 인터랙티브 미디어 아트 분야에서 다채로운 영향을 미치고 있습니다. 그의 작품과 사상은 예술의 다양성과 경계를 넘어서는 창조적 흐름의 지속적인 탐구를 강조하는 데 중요한 역할을 합니다.


In [ ]:
for snode in response.source_nodes:
  print(snode.node.get_content())
  print('*' * 30)

또한 백남준은 같은 해 베니스 비엔날레 국가전시관 부문에 한국관을 설치하는 일에 결정적인 역할을 했다. 이로써 한국 미술이 세계 미술계에 진출하는 교두보가 마련되었다고 하겠다. 같은 해 그의 예술적 정수가 담긴 일렉트로닉 수퍼하이웨이 전시를 진행했다. 1996년 4월 9일 뇌졸중으로 쓰러졌으며, 6개월만인 그해 10월에 재기했다. 2000년 뉴욕 구겐하임 미술관에서 ‘백남준의 세계’ 라는 대규모 회고전이 열렸으며, 이때 백남준은 레이저 아트 ‘야곱의 사다리’, ‘삼원소’ 등을 전시한 바 있다.
2006년 1월 29일, 미국 마이애미의 자택에서 노환으로 75세로 별세, 유해가 서울, 뉴욕, 독일에 나눠서 안치되었다..
******************************
이 영감은 "세계의 역사는 우리에게 알려준다. 주어진 게임에서 이길 수 없다면 규칙을 바꿔라"라는 것으로 규정된다. 이후 1950년대부터 활발해지기 시작한 독일 라인 지역의 액션뮤직의 현장에서 백남준은 ‘아시아에서 온 문화테러리스트’(앨런 카프로)라고 불릴 정도의 탁월한 퍼포먼스 아티스트로 활약했다. 1959년 ‘존 케이지에게 보내는 경의’에서 음악적 콜라주와 함께 피아노를 부수는 퍼포먼스를 선보이는 것을 시작으로, 바이올린을 단숨에 파괴하거나(바이올린 솔로) 존 케이지가 착용한 넥타이를 잘라버리는 퍼포먼스(피아노 포르테를 위한 연습곡)가 특히 유명하다. 이 초기 퍼포먼스에 대해 백남준은 스스로 "충격, 표현주의, 낭만주의, 클라이맥스, 놀라움, 기타 등등을 보여준 것"이라고 표현한 바 있다.
******************************
졸업 논문은 ‘아르놀트 쇤베르크 연구’이다.
1956년 백남준은 졸업과 함께 독일로 유학을 떠나 뮌헨 대학교 및 쾰른 대학교 등에서 서양의 건축, 음악사, 철학 등을 공부하였다. 뮌헨 대학교 입학 1년 후에는 프라이부르크 국립 음악 대학교로 옮겨 볼프강 포르트너 교수에게 배우지만, 곧 쇤베르크 이후 현대음악의 실험이 활발히 진행되던 다름슈타트 하기 

In [ ]:
response = query_engine.query("맥스웰 방정식이 무엇인가요?")
print(response)

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


맥스웰 방정식은 영국의 물리학자 제임스 클러크 맥스웰이 19세기 중반에 발전시킨 이론으로, 전기장과 자기장의 행동을 포괄적으로 설명하는 네 가지 주요 방정식으로 구성되어 있습니다:

1. **전기장의 보존 법칙 (일적분 형태의 법칙)**: 이 방정식은 시간에 따른 전기장의 변화율이 전하 밀도 벡터 \(\rho\)와 진공 전기 허용률 \(\epsilon_0\)의 곱에 비례한다는 것을 나타냅니다.
   \[ \nabla \cdot \mathbf{E} = \frac{\rho}{\epsilon_0} \]
   여기서 \(\nabla \cdot\)는 발산 연산자입니다.

2. **자기장의 생성과 변화**: 변화하는 전기장이 자기장을 만들어낼 수 있다는 원리를 설명하며, 이 상호작용을 수학적으로 정식화합니다. 이는 파라데이 법칙 또는 맥스웰 유도 법칙으로 알려져 있습니다:
   \[ \nabla \times \mathbf{E} = -\frac{\partial \mathbf{B}}{\partial t} \]
   여기서 \(\nabla \times\)는 회전 연산자를 나타내며, \(\mathbf{B}\)는 자기장 벡터입니다.

3. **전류와 시간변화되는 전기장에 의한 자장 생성** (마이스너-맥스웰 법칙): 이 법칙은 전류 밀도 \(\mathbf{J}\)와 교류 전기장에 의해 생성되는 유도 자기장 사이의 관계를 명확히 합니다.
   \[ \nabla \times \mathbf{B} = \mu_0 \mathbf{J} + \mu_0 \epsilon_0 \frac{\partial \mathbf{E}}{\partial t} \]
   여기서 \(\mu_0\)는 진공의 자기 허용률입니다.

4. **전하 보존과 관련된 또 다른 변환 법칙**: 이 방정식은 전자기 유도의 개념을 포괄하며, 변화하는 자기장이 전기장을 생성한다는 입혔습니다. 여기서 \(\mu_0\)는 여전히 진공의 자기 허용률을 나타냅니다.
   \[ \nabla \times \mathbf{B} = \mu_0 \ma

In [ ]:
for snode in response.source_nodes:
  print(snode.node.get_content())
  print('*' * 30)

자기에서 전기가 유도될 수 있다는 것이 알려지자, 이것을 이용한 많은 기술들이 발명되었다. 니콜라 테슬라는 전자기 유도를 이용하여 교류 발전기를 발명하였다.
맥스웰 방정식.
1864년 제임스 클러크 맥스웰은 기존의 전기와 자기에 대한 이론의 수리 모형들을 약 20개의 방정식으로 정리하였다. 맥스웰은 나중에 의미가 같은 방정식들은 과감히 생략하고 본질적인 네 개의 방정식으로 정리하였다. 이를 맥스웰 방정식이라고 한다.
맥스웰 방정식이 다루는 전자기학의 법칙은 다음과 같다.
맥스웰은 전자기학에 대한 법칙들의 수리 모형을 정리하면서 로런츠 힘은 다루지 않았다. 로렌츠 힘은 전하를 띈 물체가 전기장 안에서 받는 힘을 뜻한다. 오늘날 전자기학은 맥스웰 방정식과 로런츠 힘을 기반으로 하고 있다.
맥스웰은 맥스웰 방정식을 정리한 후, 자계가 사인파와 같은 파동을 보이며 변화한다면 전계 역시 그에 따라 사인파 파동을 만들어 내게 될 것이고, 그 반대도 마찬가지 일 것이므로, 전기와 자기의 성질을 모두 갖는 파동, 즉 전자파가 존재할 것이라고 예측하였다.
******************************
그러나 아인슈타인은 사이언스에의 기고문에서 이를 "맥스웰 방정식"이라 부르며, 이 방정식들이 이론물리학의 기초라고 설명하였다. 맥스웰은 방정식을 정리하면서 헤비사이드의 전위와 벡터 위치 등 위치 요소를 중요한 개념으로 도입하였다. 1884년 맥스웰은 전자기파의 전달을 중력과 같이 원격에서 상호작용하는 힘이 아닌 전자기장에서 빛의 속도로 전파되는 전위로 파악하였다. 라디오 안테나에 대한 현대의 분석에서도 맥스웰의 백터와 스칼라 위키에 대한 수식만으로 서로 떨어져 있는 안테나 사이에 작용하는 전파의 영향을 모두 설명할 수 있다.
맥스웰 방정식과 관련한 헤비사이드의 업적은 맥스웰이 여러 논문과 책에서 서술한 맥스웰 방정식을 오늘날과 같은 4개의 방정식으로 정리하였다는 것이다.
《물리적 역선에 대해》 (1861년).
오늘날 4개의 방정식으로 정리된 맥스웰의 방정식은 1861년 

In [ ]:
from llama_index.core.query_engine import SubQuestionQueryEngine## 꼬리질문 만들수 있음
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="wiki_docs",
            description=""
        ),
    ),
]

sub_query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
)

In [ ]:
sub_query_prompts = sub_query_engine.get_prompts()#유저 질문에 관련된 서브 퀘스천 만들어 달라

In [ ]:
print(sub_query_prompts['question_gen:question_gen_prompt'].template)

Given a user question, and a list of tools, output a list of relevant sub-questions in json markdown that when composed can help answer the full user question:

# Example 1
<Tools>
```json
{
    "uber_10k": "Provides information about Uber financials for year 2021",
    "lyft_10k": "Provides information about Lyft financials for year 2021"
}
```

<User Question>
Compare and contrast the revenue growth and EBITDA of Uber and Lyft for year 2021


<Output>
```json
{
    "items": [
        {
            "sub_question": "What is the revenue growth of Uber",
            "tool_name": "uber_10k"
        },
        {
            "sub_question": "What is the EBITDA of Uber",
            "tool_name": "uber_10k"
        },
        {
            "sub_question": "What is the revenue growth of Lyft",
            "tool_name": "lyft_10k"
        },
        {
            "sub_question": "What is the EBITDA of Lyft",
            "tool_name": "lyft_10k"
        }
    ]
}
```

# Example 2
<Tools>
```json
{

In [ ]:
response = sub_query_engine.query("백남준은 누구인가요?")

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Generated 3 sub questions.
[wiki_docs] Q: 백남준의 생애와 주요 작품 소개는 무엇인가요?


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

...

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[wiki_docs] A: 
백남준은 직접적으로 노벨 물리학상이나 주요 노벨 문학상과 같은 공식적인 과학 및 국제 문학상 분야에서 수상한 기록이 없습니다. 그러나 그의 문화적 및 예술적 업적은 매우 높이 평가받고 있습니다. 백남준은 1983년 영어권 작가에게 수여되는 영국 맨 부커 상을 수상하였으며, 이어서 1999년에 두 번째로 이 상을 받았습니다. 이 업적으로 그는 국제적으로 널리 알려져 있었고, 이러한 예술적 성과가 그에게 더 큰 인정을 가져다 준 것으로 보입니다. 

그러나 만약 질문이 정확히 어떤 분야의 수상 경력을 묻는 것이라면, 현재 공지된 정보 내에서는 그가 노벨 상의 직접적 수상자라기 보다는 특정 문학상 분야에서의 다재다능한 업적을 인정받았다는 점을 강조할 수 있습니다. 그의 가장 유명한 상이 맨 부커 상으로, 이는 그의 예술적 성취가 국제적으로 어떻게 인정받았는지를 보여주는 중요한 지표입니다. 노벨 문학상 수상에 이르는 구체적인 과정이나 상은 다를 수 있으니, 각 상의 의미와 배경에 대해 더 자세히 알고 싶으시다면 추가 조사가 필요할 수 있습니다.


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [ ]:
response = sub_query_engine.query(
    "맥스웰 방정식이 무엇인가요?"
)

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Generated 3 sub questions.
[wiki_docs] Q: 맥스웰 방정식은 어떤 현상을 설명하는가?


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

...

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[wiki_docs] A: 
닉 맥스웰 방정식의 역사적 배경을 이해하기 위해서는 다음의 세부 사항들을 고려해야 합니다:

1. **기초 발견의 시간**:
   - **초기 이론적 기반**: 맥스웰 방정식의 역사적 해석을 이해하는 첫걸음은 전기와 자기 현상에 대한 초기 이론적 탐구로 거슬러 올라갑니다. 18세기 후반부터 전기학자들인 마이클 패러데이와urgical 인 John C. Loudon은 전기와 자기장의 상호작용을 탐색하였습니다. 특히 패러데이는 전기 유도 현상을 발견함으로써 자기장과 전기의 관계를 규명하는데 핵심적인 기여를 했습니다.
   
2. **맥스웰의 통합 작업**:
   - **체계적 접근**: 19세기 후반, 제임스 클러크 맥스웰은 이들 분야의 연구를 종합하여 20세기 초 전기자기의 기초를 이루는 네 가지 편미분 방정식으로 체계화했습니다. 그의 방정식들은 두 물리 법칙, 즉 쿨롱 법칙과 앰페르의 법칙을 하나로 통합시켰습니다. 맥스웰은 이 방정식들이 자연 현상의 모든 측면을 설명할 수 있다는 강력한 이론을 제시하며, 전기공학과 현대 물리학의 금자탑 같은 업적을 이루었습니다.
   - **발표와 전파**: 맥스웰의 이론은 처음에 학술적 논문들을 통해 발표되었으나, 그의 통찰은 전통적인 기학적 관점을 넘어서 상대성 이론과 양자 역학 등 현대 물리학의 발전에 핵심적인 역할을 했습니다.

3. **수학적 및 과학적 영향**:
   - **수학적 체계화**: 맥스웰 방정식은 현대 수학적 기법들을 도입하고 활용하여 물리학의 이론적 기반을 특별법화함으로써, 물리학 내에서뿐만 아니라 공학 분야에도 혁명적인 변화를 초래했습니다. 이를 통해 전기와 자기 현상은 더 정확하게 예측 및 설명될 수 있었고, 이후 라디오, 텔레그라피, 전자기파의 자체 연구 등 다양한 기술 발전의 토대가 되었습니다.
   - **헤비사이드의 영향**: 리처드 헤비사이드와 같은 과학자들은 맥스웰 방정식을 더욱 시대적 맥락에 맞게 재해석하고 정리하여, 이를 통해 미분기하학 및 이론 물리학의 학

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]